In [ ]:
# default_exp followers

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#hide
!pip install -qU nbdev
!pip install -qU fastcore

In [ ]:
#hide
from pathlib import Path
path = Path("/content/drive/My Drive/upwork/Martin followers/tomorrowswig_followers")
%cd "{path}"

/content/drive/My Drive/upwork/Martin followers/tomorrowswig_followers


In [ ]:
#hide
#not deps but we need them to use nbdev and run tests
from nbdev import * 
from nbdev.showdoc import *
from fastcore.test import *

# Instagram followers
> Here I'll create a script to fetch new followers for an instagram account. The reason is to track followers change by country, facebook API offers only lifetime numbers which are updated daily.

https://developers.facebook.com/tools/explorer/

https://developers.facebook.com/docs/facebook-login/access-tokens/

## Facebook Auth

In [ ]:
#hide
!pip install -qU gspread
!pip install -qU python-facebook-api

In [ ]:
#hide
#test_flag_colab
%%capture

import yaml
with open(path.parent/"facebook_creds.yml") as f:
    data = yaml.safe_load(f)
    %env APP_ID {data.get("APP_ID")}
    %env APP_SECRET {data.get("APP_SECRET")}
    %env TOKEN {data.get("TOKEN")}
    %env USER_ID {data.get("USER_ID")}

In [ ]:
#export
from datetime import datetime
from typing import *

from pyfacebook import IgProApi
from pyfacebook.error import *
from tomorrowswig_followers.core import *

In [ ]:
APP_ID = os.environ.get("APP_ID")
APP_SECRET = os.environ.get("APP_SECRET")
TOKEN = os.environ.get("TOKEN")
USER_ID = os.environ.get("USER_ID")

## Get new followers with graph api

In [ ]:
#hide
!curl -i -X GET \
 "https://graph.facebook.com/v7.0/17841409206135869/insights?metric=audience_country&period=lifetime&access_token={TOKEN}"

HTTP/2 200 
vary: Accept-Encoding
etag: "b4382957f4f0d4e4126c7a1d1eac1bdb82dbd765"
x-business-use-case-usage: {"17841409206135869":[{"type":"instagram","call_count":1,"total_cputime":1,"total_time":1,"estimated_time_to_regain_access":0}]}
content-type: application/json; charset=UTF-8
facebook-api-version: v7.0
strict-transport-security: max-age=15552000; preload
pragma: no-cache
x-fb-rev: 1002435395
access-control-allow-origin: *
cache-control: private, no-cache, no-store, must-revalidate
x-fb-trace-id: CWtMs8+FFsu
x-fb-request-id: Ay5ugzVubqkL32cKGIAgDvx
expires: Sat, 01 Jan 2000 00:00:00 GMT
x-fb-debug: kuTzc/4TVLxVqBu2L9zMmR2eWeDrdepsIZ/N7+FN88oxgU8UQ4fIsI6nHnzqGOB+JYJuAuum+8eFw/b5NQ1Gxg==
date: Wed, 29 Jul 2020 16:20:10 GMT
alt-svc: h3-29=":443"; ma=3600,h3-27=":443"; ma=3600

{"data":[{"name":"audience_country","period":"lifetime","values":[{"value":{"RS":56,"DE":866,"NP":37,"BD":54,"TW":124,"PT":155,"JO":41,"JP":82,"UA":40,"FR":57,"HU":99,"BR":2306,"MA":88,"SG":618,"DZ":105,"KE":

In [ ]:
api = IgProApi(app_id=APP_ID, app_secret=APP_SECRET, long_term_token=TOKEN,
               version="5.0")

AccessToken(app_id='556739541150472', application='Wells Hampstead', user_id='2522505204728318')

In [ ]:
try:
    response = api.get_user_insights(user_id=USER_ID, period="lifetime", metrics=["audience_country"],
                                    return_json=True)
except PyFacebookException as e:
    display(e.message)

In [ ]:
response = response[0]["values"][0]

In [ ]:
response["end_time"]

'2020-07-29T07:00:00+0000'

In [ ]:
#export
def get_followers() -> Tuple[str, Dict[str, int]]:
    api = IgProApi(
        app_id=APP_ID, app_secret=APP_SECRET, long_term_token=TOKEN, version="5.0"
    )
    api.get_token_info()
    try:
        response = api.get_user_insights(
            user_id=USER_ID,
            period="lifetime",
            metrics=["audience_country"],
            return_json=True,
        )[0]["values"][0]
        return response["end_time"].split("T")[0], response["value"]
    except PyFacebookException as e:
        return (e.message, {0: 0})

## Google Sheets

### Insert new followers to a google sheet 

In [ ]:
end_time, followers = get_followers()

In [ ]:
df = get_df("History")

In [ ]:
end_time = datetime.strptime(end_time, "%Y-%m-%d").strftime("%b \t %d")

In [ ]:
new_followers = pd.Series(followers)
new_followers.name = f"{end_time} {str(datetime.utcnow()).split('.')[0]}"
if new_followers.name not in df.columns:
    df = pd.concat([df, new_followers], axis=1)
    df = df[[df.columns[-1]] + df.columns[:-1].tolist()]
df.sort_index(inplace=True)
df = df.fillna(0).astype(int)

In [ ]:
#export
def get_updated_followers(
    df: pd.DataFrame, data: Dict[str, int], end_time: str
) -> pd.DataFrame:
    new_followers = pd.Series(data)
    end_time = datetime.strptime(end_time, "%Y-%m-%d").strftime(f"%b %d{' '*16}")
    new_followers.name = f"{end_time} {str(datetime.utcnow()).split('.')[0]}"
    if new_followers.name not in df.columns:
        df = pd.concat([df, new_followers], axis=1)
        df = df[[df.columns[-1]] + df.columns[:-1].tolist()]
    df.sort_index(inplace=True)
    return df.fillna(0).astype(int)

In [ ]:
fetched_followers = {'AR': 1001, 'AU': 724, 'BD': 55, 'BR': 2250, 'CA': 1680, 'CL': 526, 'CO': 6}
end_time = "2020-07-21"

test_followers_df = get_updated_followers(pd.DataFrame(), fetched_followers, end_time)
test_followers_df.rename(columns={test_followers_df.columns[0]: "2020-01-01" }, inplace=True)
assert {'2020-01-01': fetched_followers} == test_followers_df.to_dict()

test_followers_df = get_updated_followers(test_followers_df, fetched_followers, end_time)
test_followers_df.rename(columns={test_followers_df.columns[0]: "2020-01-02" }, inplace=True)
assert {'2020-01-02': fetched_followers,
        '2020-01-01': fetched_followers} == test_followers_df.to_dict()

In [ ]:
#export
def save_followers() -> None:
    df = get_df("History")
    end_time, followers = get_followers()
    df = get_updated_followers(df, followers, end_time)
    write_df(df, "History")

In [ ]:
# save_followers()

### Get difference from the previous day

In [ ]:
wsh = SH.worksheet("History")
df = pd.DataFrame(wsh.get_all_records())
df.set_index(df["countries"], inplace=True)
df.drop(columns="countries", inplace=True)

In [ ]:
df.head()

,Jul 29 2020-07-29 13:10:41,Jul 29 2020-07-29 07:10:37,Jul 28 2020-07-29 01:10:35,Jul 28 2020-07-28 19:10:33,Jul 28 2020-07-28 13:10:37,Jul 28 2020-07-28 07:10:44,Jul 27 2020-07-28 01:10:39,Jul 27 2020-07-27 19:10:35,Jul 27 2020-07-27 13:10:34,Jul 27 2020-07-27 07:10:41,Jul 26 2020-07-27 01:10:34,Jul 26 2020-07-26 19:10:34,Jul 26 2020-07-26 13:10:33,Jul 26 2020-07-26 07:10:33,Jul 25 2020-07-26 01:10:37,Jul 25 2020-07-25 19:10:35,Jul 25 2020-07-25 13:10:42,Jul 25 2020-07-25 07:10:37,Jul 24 2020-07-25 01:10:41,Jul 24 2020-07-24 19:10:41,Jul 24 2020-07-24 13:10:42,Jul 24 2020-07-24 07:10:34,Jul 23 2020-07-24 01:10:38,Jul 23 2020-07-23 19:10:30,Jul 23 2020-07-23 13:11:01,Jul 23 2020-07-23 07:10:57,Jul 22 2020-07-23 01:10:56,Jul 22 2020-07-22 19:11:01,Jul 22 2020-07-22 13:11:00,Jul 22 2020-07-22 07:11:00,Jul 21 2020-07-22 01:11:01,Jul 20 2020-07-21 01:11:01,Jul 19 2020-07-20 00:14:39,Jul 19 2020-07-19 17:46:29,Jul 18 2020-07-19 00:23:09,Jul 17 2020-07-17 21:47:01,"July 16?""",Jul 15,Jul 13,Jul 12,...,Aug-6,Aug-5,Aug-4,Aug-3,Aug-2,Aug-1,Jul-31,Jul-30,Jul-29,Jul-28,Jul-27,Jul-25,Jul-24,Jul-23,Jul-22,Jul-21,Jul-18,Jul-17,Jul-16,Jul-15,Jul-14,Jul-13,Jul-12,Jul-11,Jul-10,Jul-9,Jul-8,Jul-7,Jul-5,Jul-3,Jun-28,Jun-26,Jun-25,Jun-24,Jun-23,Jun-22,Jun-21,Jun-20,Jan-19,Jan-18
countries,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AR,1048,1048,1048,1048,1028,1028,1028,1026,1026,1026,1026,1026,1027,1027,1027,1027,1028,1028,1028,1024,1024,1024,1024,1024,1012,1012,1012,1000,1000,1000,1000,1001,1003,1004,1004,1009,1009,1005,985,967,...,239,240,241,241,241,241,242,242,244,244,244,244,246,245,245,245,250,250,250,250,250,250,251,253,253,254,254,254,255,256,256,257,258,233,218,200,145,140,247,248
AU,794,794,794,794,787,787,787,752,752,752,752,752,738,738,738,738,739,739,739,724,724,724,724,724,724,724,724,724,724,724,724,724,711,708,708,709,709,705,708,707,...,51,50,51,51,45,42,0,42,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,39,39,39,40,40,40,40,40,41,40,0,0
BD,54,54,54,54,54,54,54,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,54,54,54,54,54,54,53,54,...,79,79,80,80,80,81,81,81,81,81,82,82,81,81,81,81,83,83,83,83,83,83,84,85,85,85,85,85,86,86,87,87,87,86,88,89,84,76,82,83
BR,2306,2306,2306,2306,2301,2301,2301,2272,2272,2272,2272,2272,2243,2243,2243,2243,2244,2244,2244,2235,2235,2235,2235,2235,2243,2243,2243,2247,2247,2247,2247,2250,2221,2196,2196,2194,2194,2180,2183,2185,...,873,863,852,831,818,810,791,775,760,742,731,684,669,654,644,627,595,566,547,528,504,475,459,434,404,374,354,351,337,325,291,278,271,259,248,239,164,159,618,603
CA,1729,1729,1729,1729,1727,1727,1727,1693,1693,1693,1693,1693,1682,1682,1682,1682,1683,1683,1683,1670,1670,1670,1670,1670,1673,1673,1673,1676,1676,1676,1676,1680,1671,1664,1664,1671,1671,1662,1667,1668,...,113,109,105,104,104,102,98,94,91,90,86,81,81,77,76,74,69,66,63,59,59,59,60,60,60,59,58,58,59,59,57,55,55,55,56,56,35,35,71,71


In [ ]:
change_df = df.diff(axis=1, periods=-1)

In [ ]:
change_df = change_df.fillna(0).astype(int)

In [ ]:
change_df = change_df.applymap(lambda x: f"+{x}" if x >0 else x)

In [ ]:
change_df = "(" + change_df.astype(str) + ")" + " " + df.astype(str)

In [ ]:
change_df = change_df.replace(["\(0\) 0", "\(0\) "], "", regex=True)

In [ ]:
change_history = get_worksheet("Change History")

In [ ]:
#export
def save_change():
    df = get_df("History")
    change_df = make_change(df)
    write_df(change_df, "Change History")

In [ ]:
#export
def make_change(df: pd.DataFrame) -> pd.DataFrame:
    change_df = df.diff(axis=1, periods=-1)
    change_df = change_df.fillna(0).astype(int)
    change_df = change_df.applymap(lambda x: f"+{x}" if x > 0 else x)
    change_df = "(" + change_df.astype(str) + ")" + " " + df.astype(str)
    return change_df.replace(["\(0\) 0", "\(0\) "], "", regex=True)

In [ ]:
test_change_df = make_change(pd.DataFrame({"2020-07-21": [1001, 724, 55, 2250, 1680],
                                           "2020-07-20": [1004, 708, 55, 2300, 1600]},
                                          index=["AR", "AU", "BD", "BR", "CA"]))
assert test_change_df.iloc[4,:].to_list() == ["(+80) 1680", "1600"]
assert test_change_df.iloc[3,:].to_list() == ["(-50) 2250", "2300"]

In [ ]:
# save_change()

In [ ]:
#export
def lambda_handler(
    event: Dict = None, context=None,
):
    """Event and context are for AWS Lambda"""
    get_followers()
    save_followers()
    save_change()
    return "Success"

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_followers.ipynb.
Converted 02_ads.ipynb.
Converted index.ipynb.
